# Investigation of the Winner's Curse effect in the German Property Market

## Introduction

The Winner's Curse Effect is present in almost any auction. Because items (i.e. properties) are each unique, there is no market value to which they could be compared. Therefore, each bidder assesses the value independently. Different bidders have different perceptions about the value of the item. Some of them might be more optimistic while others tend to be less so. Because we are in an auction setting, the highest bidder (i.e. the most optimistic one) wins.

If we assume that there is some Wisdom of the Crowd effect, the average perceived value will be the acutal value of the object. Then, assuming variability in the bids, it is very likely that the winner of the auction will overpay.

## Methodology



## Code

In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os

In [2]:
# get the html file of a webpage
def get_html(url):
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser")

In [37]:
def extract_data(html, write=None, output=False, next=False):
    Frame = pd.DataFrame()
    
    # loop to get all the objects from one page.
    for object in html.find_all(class_ = "tx_goauktion_block"):

        nr = object.find(class_="h-col katalogpos").find("p").text

        status = object.find(class_="h-col verkaufsstatus").find("p").text
        
        prices = object.findAll(class_ = "d-col auktion_limit price sold")
        limit_price = prices[0].find("p").text.strip()
        sold_at = prices[1].find("p").text
        
        info = object.find(class_="addr-list")
        info = " ".join(info.find("a").text.split())
        
        data = {"info": info, "sold_at": sold_at, "status": status, "limit_price": limit_price, "nr": nr}
        # data = [nr, status, limit_price, sold_at, info]
        Frame = Frame.append(data, ignore_index=True)

    last = False
    # if this fails, we have reached the last page.
    try:
        next_link = html.find(class_="next").find("a", href=True)["href"]
    except:
        last = True
        next_link = None

    
    # check if user wants to write the data to a csv file
    if write:
        Frame.to_csv(write, mode="a")
    
    
    if output:
        if next:
            return last, Frame, next_link
        else:
            return last, Frame
    else:
        return last, next_link

In [38]:
def scrape(first_page):
    first_page = get_html(first_page)
    last_page, next_link = extract_data(first_page, write="data.csv", next=True)
    
    while not last_page:
        pattern = r"currentPage.+?=(\d{1,2})"
        print(last_page, "Currently scraping page: {}".format(re.findall(pattern, next_link)[0]))
        new_html = get_html("https://www.sga-ag.de/" + next_link)
        last_page, next_link = extract_data(new_html, write="data.csv", next=True)
    else: 
        print("Finished Scraping, content extracted to {}".format())

In [ ]:
scrape("https://www.sga-ag.de/ergebnisse/ergebnisliste.html?status=ergebnisliste&cHash=400a70d7f4d26f5dbd219b15bda854e5#tx-goauktion-deep")

In [39]:
extract_data(get_html("https://www.sga-ag.de/ergebnisse/ergebnisliste.html?status=ergebnisliste&tx_goauktion_soldlist%5B%40widget_0%5D%5BcurrentPage%5D=72&cHash=6068355671148448bf8c11b501d9d4dd#tx-goauktion-deep"), write="test.csv")

(True, None)

In [67]:
df_raw = pd.read_csv("data.csv")

In [68]:
df = df_raw.drop_duplicates(subset="nr").drop(df_raw.columns[0], axis=1) #.to_csv("test.csv", "w")

In [78]:
pattern = r"[A-Z](\d+)-"
re.findall(pattern, df.iloc[1]["nr"])[0]

'21'

In [82]:
df.apply(lambda x: re.findall(pattern, x)[0], axis=1, columns=["nr"])

TypeError: <lambda>() got an unexpected keyword argument 'columns'